In [2]:
!pip install "tritonclient[all]"

  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.2
    Uninstalling protobuf-5.29.2:
      Successfully uninstalled protobuf-5.29.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
skl2onnx 1.17.0 requires onnxconverter-common>=1.7.0, which is not installed.
opentelemetry-exporter-otlp-proto-grpc 1.27.0 requires opentelemetry-proto==1.27.0, which is not installed.
google-cloud-resource-manager 1.12.4 requires google-api-core[grpc]!=2.0.*,!=2.1.*,!=2.10.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,!=2.9.*,<3.0.0dev,>=1.34.1, which is not installed.
google-cloud-resource-manager 1.12.4 requires proto-plus<2.0.0dev,>=1.22.3, which is not installed.
google-generativeai 0.5.4 requires google-ai

In [3]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load an official model

# Retreive metadata during export
metadata = []


def export_cb(exporter):
    metadata.append(exporter.metadata)


model.add_callback("on_export_end", export_cb)

# Export the model
onnx_file = model.export(format="onnx", dynamic=True)

100%|██████████| 5.35M/5.35M [00:10<00:00, 520kB/s] 


Ultralytics 8.3.51 🚀 Python-3.11.10 torch-2.2.2 CPU (Apple M1)
YOLO11n summary (fused): 238 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.



PyTorch: starting from 'yolo11n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (5.4 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: slimming with onnxslim 0.1.44...
ONNX: export success ✅ 13.8s, saved as 'yolo11n.onnx' (10.1 MB)

Export complete (14.4s)
Results saved to /Users/shephardmarufu/Documents/development/experiments/gofer
Predict:         yolo predict task=detect model=yolo11n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolo11n.onnx imgsz=640 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app


In [1]:
from pathlib import Path

# Define paths
model_name = "yolo"
triton_repo_path = Path("tmp") / "triton_repo"
triton_model_path = triton_repo_path / model_name

# Create directories
(triton_model_path / "1").mkdir(parents=True, exist_ok=True)

In [3]:
tag = "nvcr.io/nvidia/tritonserver:24.09-py3"
cmd_dc=f"docker run -d --rm -v {triton_repo_path}:/models -p 8000:8000 {tag} tritonserver --model-repository=/models"
print(cmd_dc)

docker run -d --rm -v tmp/triton_repo:/models -p 8000:8000 nvcr.io/nvidia/tritonserver:24.09-py3 tritonserver --model-repository=/models


In [5]:
from pathlib import Path

# Move ONNX model to Triton Model path
Path(onnx_file).rename(triton_model_path / "1" / "model.onnx")

# Create config file
(triton_model_path / "config.pbtxt").touch()

# (Optional) Enable TensorRT for GPU inference
# First run will be slow due to TensorRT engine conversion
data = """
optimization {
  execution_accelerators {
    gpu_execution_accelerator {
      name: "tensorrt"
      parameters {
        key: "precision_mode"
        value: "FP16"
      }
      parameters {
        key: "max_workspace_size_bytes"
        value: "3221225472"
      }
      parameters {
        key: "trt_engine_cache_enable"
        value: "1"
      }
      parameters {
        key: "trt_engine_cache_path"
        value: "/models/yolo/1"
      }
    }
  }
}
parameters {
  key: "metadata"
  value: {
    string_value: "%s"
  }
}
""" % metadata[0]

with open(triton_model_path / "config.pbtxt", "w") as f:
    f.write(data)

In [6]:
import contextlib
import subprocess
import time

from tritonclient.http import InferenceServerClient

# Define image https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tritonserver
tag = "nvcr.io/nvidia/tritonserver:24.09-py3"  # 8.57 GB

# Pull the image
subprocess.call(f"docker pull {tag}", shell=True)

# Run the Triton server and capture the container ID
container_id = (
    subprocess.check_output(
        f"docker run -d --rm -v {triton_repo_path}:/models -p 8000:8000 {tag} tritonserver --model-repository=/models",
        shell=True,
    )
    .decode("utf-8")
    .strip()
)

# Wait for the Triton server to start
triton_client = InferenceServerClient(url="localhost:8000", verbose=False, ssl=False)

# Wait until model is ready
for _ in range(10):
    with contextlib.suppress(Exception):
        assert triton_client.is_model_ready(model_name)
        break
    time.sleep(1)

Cannot connect to the Docker daemon at unix:///Users/shephardmarufu/.docker/run/docker.sock. Is the docker daemon running?
docker: Cannot connect to the Docker daemon at unix:///Users/shephardmarufu/.docker/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.


CalledProcessError: Command 'docker run -d --rm --gpus 0 -v tmp/triton_repo:/models -p 8000:8000 nvcr.io/nvidia/tritonserver:24.09-py3 tritonserver --model-repository=/models' returned non-zero exit status 125.